In [41]:
from sklearn.datasets        import fetch_mldata
from sklearn.model_selection import train_test_split

# Download the MNIST dataset
mnist    = fetch_mldata('MNIST original', data_home='.')
features = mnist.data
labels   = mnist.target

# Split train and test data
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)


In [61]:
# Import DL libraries
import numpy as np
import cntk  as c

# Display CNTK version
' '.join([
    c.__name__.upper(),
    c.__version__,
    str(c.device.all_devices()[0])
])

'CNTK 2.0 CPU'

In [99]:
class ConvNet():
    IMAGE_SHAPE        = (32, 32, 1)
    IMAGE_CLASSES      = [n for n in range(0, 10)]
    IMAGE_CLASS_COUNT  = len(IMAGE_CLASSES)
    LEARNING_RATE      = 0.001
    EPOCH_COUNT        = 20
    BATCH_SIZE         = 64
    DROP_RATE          = 0.5
    HIDDEN_LAYER_COUNT = 96
    
    def __init__(self):
        self._build_inputs()
        self._build_layers()
        self._build_stack()
        self._build_trainer()
    
    def evaluate(self, fn, feeds):
        c.logging.log_number_of_parameters(self.stack); print()
        for epoch in range(self.EPOCH_COUNT):
            batch = self.batch(epoch, feeds)
            fn(epoch, batch)
    
    def train(self, data):
        self.trainer.train_minibatch(data)
    
    def summarize(self):
        self.trainer.summarize_training_progress()
    
    def checkpoint(self, version):
        self.stack.save(
            os.path.join(
                '.', '_'.join(
                    'ConvNet', 'MNIST', '{}.dnn'.format(version)
                )
            )
        )
    
    def batch(self, epoch, data):
        def chunk(data):
            slice_begin = epoch * self.BATCH_SIZE
            slice_end   = batch_begin + self.BATCH_SIZE
            return data[slice_begin:slice_end]
        return {key: chunk(value) for key, value in data}
    
    def _build_inputs(self):
        self.inputs = c.input_variable(self.IMAGE_SHAPE,       np.float32, name='inputs')
        self.labels = c.input_variable(self.IMAGE_CLASS_COUNT, np.float32, name='labels')
    
    def _build_layers(self):
        with c.layers.default_options(activation=c.ops.relu, pad=False):
            self.layers  = [
                c.layers.Convolution2D((5,5), 32, pad=True),
                c.layers.MaxPooling((3,3), (2,2)),
                c.layers.Convolution2D((3,3), 48),
                c.layers.MaxPooling((3,3), (2,2)),
                c.layers.Convolution2D((3,3), 64),
                c.layers.Dense(self.HIDDEN_LAYER_COUNT),
                c.layers.Dropout(self.DROP_RATE),
                c.layers.Dense(self.IMAGE_CLASS_COUNT, activation=None)
            ]
    
    def _build_stack(self):
        self.stack = self.inputs
        for layer in self.layers: self.stack = layer(self.stack)
        self.loss  = c.losses.cross_entropy_with_softmax(self.stack, self.labels)
        self.error = c.metrics.classification_error(self.stack, self.labels)
    
    def _build_trainer(self):
        learner      = c.learners.sgd(self.stack.parameters, self.LEARNING_RATE)
        printer      = c.logging.ProgressPrinter(tag='Training', num_epochs=self.EPOCH_COUNT)
        self.trainer = c.Trainer(self.stack, (self.loss, self.error), learner, printer)
